<a href="https://colab.research.google.com/github/Sanjichka/RAG/blob/main/RAG_GitHub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U langchain-google-genai langchain-chroma langchain-community
!pip install python-dotenv unstructured python-docx

import os
import shutil
import hashlib
from typing import List
from dotenv import load_dotenv

from docx import Document as DocxDocument
from langchain_core.documents import Document
from langchain_chroma import Chroma

from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate

load_dotenv()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.1/719.1 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.9/234.9 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.8/167.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 76.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.0/329.0 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 6.2 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=5f3f46971d70f27807316dc089e5c57f1a674462661abaf7b5a71c852cb4eacf
  Stored in directory: /root/.cache/pip/wheels/c1/67/88/e844b5b022812e15a52e4eaa3

False

In [2]:
CHROMA_PATH = "chroma"
DATA_PATH = "data"
os.environ["GOOGLE_API_KEY"] = ""

In [3]:
def get_embedding_function():
    return GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

In [4]:
def load_documents():
    documents = []

    for filename in os.listdir(DATA_PATH):
        if filename.endswith(".docx"):
            full_path = os.path.join(DATA_PATH, filename)
            docx = DocxDocument(full_path)

            current_title = "General"  # Fallback title
            current_content = ""

            for para in docx.paragraphs:
                # Detect Word Heading styles
                if para.style.name.startswith("Heading"):
                    if current_content.strip():
                        documents.append(Document(
                            page_content=current_content.strip(),
                            metadata={
                                "title": current_title,
                                "source": filename
                            }
                        ))
                    current_title = para.text.strip()
                    current_content = ""
                else:
                    # Collect text under the current heading
                    if para.text.strip():
                        current_content += para.text + "\n"

            # Append the final section of the document
            if current_content.strip():
                documents.append(Document(
                    page_content=current_content.strip(),
                    metadata={
                        "title": current_title,
                        "source": filename
                    }
                ))

    return documents

In [5]:
def hash_content(text: str) -> str:
    return hashlib.sha256(text.encode()).hexdigest()

In [6]:
# Add unique ID to each text chunk.
chunks = load_documents()

def calculate_chunk_ids(chunks: List[Document]):
    last_page_id = None
    current_chunk_index = 0

    for idx, chunk in enumerate(chunks):
        source = chunk.metadata.get("source", "unknown")
        chunk_id = f"{source}:{idx}"
        chunk.metadata["id"] = chunk_id
        chunk.metadata["content_hash"] = hash_content(chunk.page_content)
    return chunks

chunks_with_ids = calculate_chunk_ids(chunks)

In [7]:
print(chunks[1])

page_content='Understanding how BigQuery handles data is crucial for efficient analysis.
Separation of Compute and Storage: This allows you to store vast amounts of data without over-provisioning compute power, keeping costs manageable.
Dremel Query Engine: BigQuery uses the Dremel engine to execute queries in parallel across thousands of workers.
Columnar Storage: Data is stored by column rather than by row, significantly reducing the amount of data read during a query and improving performance for large-scale analytical tasks.' metadata={'title': '2. Core Architecture and Data Processing', 'source': 'BigQuery for Large-Scale Healthcare Analytics.docx', 'id': 'BigQuery for Large-Scale Healthcare Analytics.docx:1', 'content_hash': '5537cc60e7f7c319b4b2a71c1ed045fa94159e1469e789581b6a20a0ff01eecb'}


In [8]:
print(chunks_with_ids)

[Document(metadata={'title': '1. Introduction to Google BigQuery', 'source': 'BigQuery for Large-Scale Healthcare Analytics.docx', 'id': 'BigQuery for Large-Scale Healthcare Analytics.docx:0', 'content_hash': '7fe2f5dcf137cac098c455c38979b4b0bd8e6ccde3b012889007d9eea92cf291'}, page_content='Google BigQuery is a serverless, highly scalable data warehouse designed for high-performance analytics on massive datasets. Unlike traditional relational databases (OLTP), BigQuery is optimized for Online Analytical Processing (OLAP), allowing you to run complex SQL queries on petabytes of data in seconds. Its core architecture separates compute and storage, which provides the flexibility to scale resources independently as your data needs grow.'), Document(metadata={'title': '2. Core Architecture and Data Processing', 'source': 'BigQuery for Large-Scale Healthcare Analytics.docx', 'id': 'BigQuery for Large-Scale Healthcare Analytics.docx:1', 'content_hash': '5537cc60e7f7c319b4b2a71c1ed045fa94159e1

In [9]:
def add_to_chroma(chunks: List[Document]):

    db = Chroma(
        persist_directory=CHROMA_PATH,
        embedding_function=get_embedding_function()
    )

    chunks_with_ids = calculate_chunk_ids(chunks)

    # Get existing data
    existing_items = db.get(include=["metadatas"])
    existing_ids = set(existing_items["ids"])
    existing_metadatas = existing_items["metadatas"]

    # Map existing ID to its hash for comparison
    existing_hashes = {
        id_: metadata.get("content_hash")
        for id_, metadata in zip(existing_items["ids"], existing_metadatas)
    }

    print(f"Number of existing documents in DB: {len(existing_hashes)}")

    # Only add chunks if the ID is new OR the content (hash) has changed
    new_chunks = []
    for chunk in chunks_with_ids:
        chunk_id = chunk.metadata["id"]
        chunk_hash = chunk.metadata["content_hash"]

        if chunk_id not in existing_hashes or existing_hashes[chunk_id] != chunk_hash:
            new_chunks.append(chunk)

    if new_chunks:
        print(f"👉 Adding/Updating {len(new_chunks)} chunks to Gemini index...")
        new_chunk_ids = [chunk.metadata["id"] for chunk in new_chunks]
        db.add_documents(new_chunks, ids=new_chunk_ids)
        # Note: In recent LangChain/Chroma versions, persist() is called automatically.
    else:
        print("✅ No changes detected. Database is up to date.")


add_to_chroma(chunks)

Number of existing documents in DB: 0
👉 Adding/Updating 6 chunks to Gemini index...


In [ ]:
# wipes the vector DB stored in Chroma.

# def clear_database():
#     if os.path.exists(CHROMA_PATH):
#         shutil.rmtree(CHROMA_PATH)
#         print("Database cleared.")
#     else:
#         print("Database path not found.")

# Run this if you want to reset:
# clear_database()

In [15]:
from langchain_google_genai import ChatGoogleGenerativeAI

PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question in simple terms, be straightforward, and concise. Your answers must be based on the above context: {question}
"""

def query_rag(query_text: str):
    # 1. Prepare the DB with Gemini Embeddings
    embedding_function = get_embedding_function()
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

    # 2. Search the DB
    # Gemini 2.5 Flash/Pro are great at handling more context.
    # You could even try k=5 or k=10 if your chunks are small.
    results = db.similarity_search_with_score(query_text, k=3)

    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)

    # 3. Call the Gemini Model
    model = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash-lite",
        temperature=0 # Low temperature is best for factual RAG
    )

    response = model.invoke(prompt)
    response_text = response.content

    # Optional: Logic to normalize the response
    response_normalized = response_text.strip().lower()

    # 4. Extract Sources
    sources = [doc.metadata.get("id", None) for doc, _score in results]

    print(f"Response: {response_text}")
    print(f"Sources: {sources}")

    return response_normalized

In [16]:
query_text = input("Enter your question: ")
query_rag(query_text)

Enter your question: Which standard SQL is supported in BigQuery?
Response: BigQuery supports standard SQL:2011.
Sources: ['BigQuery for Large-Scale Healthcare Analytics.docx:3', 'BigQuery for Large-Scale Healthcare Analytics.docx:0', 'BigQuery for Large-Scale Healthcare Analytics.docx:1']


'bigquery supports standard sql:2011.'

# **TESTING**

In [23]:
# Prompt used to evaluate model responses
EVAL_PROMPT = """
Expected Response: {expected_response}
Actual Response: {actual_response}
---
(Answer with 'true' or 'false') Is the actual response semantically equivalent to the expected response, even if phrased differently?
"""

def query_and_validate(question: str, expected_response: str):
    # Get RAG response
    response_message = query_rag(question)

    if hasattr(response_message, "content"):
        response_text = response_message.content
    elif isinstance(response_message, dict):
        response_text = response_message.get("content", str(response_message))
    else:
        response_text = str(response_message)

    actual_normalized = response_text.strip() if response_text else "No response received"

    # Format the evaluation prompt
    prompt = EVAL_PROMPT.format(
        expected_response=expected_response, actual_response=actual_normalized
    )

    # Evaluate the response using ChatGoogleGenerativeAI
    model = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash-lite",
        temperature=0 # Low temperature is best for factual RAG
    )
    evaluation_results_str = model.invoke(prompt)
    evaluation_results_str = evaluation_results_str.content  # Extract content
    evaluation_results_str_cleaned = evaluation_results_str.strip().lower()

    print(prompt)

    # Output and validation
    if "true" in evaluation_results_str_cleaned:
        print("\033[92m" + f"Response: {evaluation_results_str_cleaned}" + "\033[0m")
        return True
    elif "false" in evaluation_results_str_cleaned:
        print("\033[91m" + f"Response: {evaluation_results_str_cleaned}" + "\033[0m")
        return False
    else:
        raise ValueError(
            f"Invalid evaluation result. Cannot determine if 'true' or 'false'."
        )

In [24]:
def test_positive():
    assert query_and_validate(
        question="Which standard SQL is supported in BigQuery?",
        expected_response="SQL version 2011",
    )
test_positive()

Response: BigQuery supports standard SQL:2011.
Sources: ['BigQuery for Large-Scale Healthcare Analytics.docx:3', 'BigQuery for Large-Scale Healthcare Analytics.docx:0', 'BigQuery for Large-Scale Healthcare Analytics.docx:1']

Expected Response: SQL version 2011
Actual Response: bigquery supports standard sql:2011.
---
(Answer with 'true' or 'false') Is the actual response semantically equivalent to the expected response, even if phrased differently?

Response: true


In [25]:
def test_negative_false():
    assert query_and_validate(
        question="Which standard SQL is supported in BigQuery?",
        expected_response="SQL 2010",
    )
test_negative_false()

Response: BigQuery supports standard SQL:2011.
Sources: ['BigQuery for Large-Scale Healthcare Analytics.docx:3', 'BigQuery for Large-Scale Healthcare Analytics.docx:0', 'BigQuery for Large-Scale Healthcare Analytics.docx:1']

Expected Response: SQL 2010
Actual Response: bigquery supports standard sql:2011.
---
(Answer with 'true' or 'false') Is the actual response semantically equivalent to the expected response, even if phrased differently?

Response: false


AssertionError: 

In [26]:
def test_negative_true():
    assert not query_and_validate(
        question="Which standard SQL is supported in BigQuery?",
        expected_response="SQL 2010",
    )
test_negative_true()

Response: BigQuery supports standard SQL:2011.
Sources: ['BigQuery for Large-Scale Healthcare Analytics.docx:3', 'BigQuery for Large-Scale Healthcare Analytics.docx:0', 'BigQuery for Large-Scale Healthcare Analytics.docx:1']

Expected Response: SQL 2010
Actual Response: bigquery supports standard sql:2011.
---
(Answer with 'true' or 'false') Is the actual response semantically equivalent to the expected response, even if phrased differently?

Response: false
